In [43]:
import pyspark
from pyspark.sql.functions import col, explode, arrays_zip


In [2]:
spark = pyspark.sql.SparkSession.builder \
    .appName('Lab7') \
    .config('spark.driver.extraClassPath', '/opt/sqljdbc_4.2/enu/jre8/sqljdbc42.jar') \
    .config('spark.executor.extraClassPath', '/opt/sqljdbc_4.2/enu/jre8/sqljdbc42.jar') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/23 15:15:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Spark server is accessible here (click on Spark UI):

In [3]:
spark

In [4]:
def display_head(df: pyspark.sql.DataFrame, n: int):
    return df.limit(n).toPandas().head(n)

In [39]:
df = spark.read \
      .format("json") \
      .option("multiline", "true") \
      .load("brzydki.json") \
      .select(col('features.type').alias('ftype'),
             'features.geometry.type',
             'features.properties.featureId',
             'features.properties.featureGroup',
             'features.properties.changeEventType',
             'features.properties.validFromTimestamp',
             'JobDetails.JobId',
             'JobDetails.jobName',
             'JobDetails.jobType',
             'maiaExtractProcessDetails.extractorVersion',
             'maiaExtractProcessDetails.extractionTimestamp',
             'maiaExtractProcessDetails.extractMode')

In [40]:
display_head(df, 5)

,ftype,type,featureId,featureGroup,changeEventType,validFromTimestamp,JobId,jobName,jobType,extractorVersion,extractionTimestamp,extractMode
0,"[Feature, Feature]","[Polygon, LineString]","[a6a7e7ff-71d3-e455-0a2c5c6a, 17e3f1c4-1802-49...","[Named Extent+Group+None, Network+Line+None]","[update, delete]","[2022-08-06T03:29:46Z, 2022-08-06T03:29:46Z]",345345,XSDR_344985_WGZX_2022_A008_V,Stereo,1.10,2022-08-06T03:30:35Z,Incremental Update


In [50]:
df = df.withColumn('temp', arrays_zip('ftype', 'type', 'featureId', 'featureGroup', 'changeEventType', 'validFromTimestamp')) \
              .withColumn('temp', explode('temp')) \
              .select('JobId',
                     'jobName',
                     'JobType',
                     'extractorVersion',
                     'extractionTimestamp',
                     'extractMode',
                     col('temp.ftype').alias('featureType'),
                     col('temp.type').alias('Type'),
                     col('temp.featureId').alias('featureId'),
                     col('temp.featureGroup').alias('featureGroup'))


In [51]:
display_head(df, 5)

,JobId,jobName,JobType,extractorVersion,extractionTimestamp,extractMode,featureType,Type,featureId,featureGroup
0,345345,XSDR_344985_WGZX_2022_A008_V,Stereo,1.10,2022-08-06T03:30:35Z,Incremental Update,Feature,Polygon,a6a7e7ff-71d3-e455-0a2c5c6a,Named Extent+Group+None
1,345345,XSDR_344985_WGZX_2022_A008_V,Stereo,1.10,2022-08-06T03:30:35Z,Incremental Update,Feature,LineString,17e3f1c4-1802-4952-a826-ba148d063716,Network+Line+None
